In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
import torch

In [ ]:
df = pd.read_csv("Datos_Portadas.csv")
data = pd.DataFrame(df)
print(data.describe())

In [ ]:
spanish_stopwords = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las",
    "por", "un", "para", "con", "no", "una", "su", "al", "es", "lo",
    "como", "más", "pero", "sus", "le", "ya", "o", "fue", "me", "si",
    "sin", "sobre", "este", "ya", "también", "entre", "cuando", "uno", "dos",
    "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "mas", "diez", "1",
    "2", "3", "4", "5", "6", "7", "8", "9", "10", "11","12", "13", "14", "15", "16",
    "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
    "aã", "tras", "mil", "anos", "tras", "niã", "son", "contra", "fueron", "hasta",
    "queda", "hasta", "os", "sera", "durante", "van", "han", "q1", "q2", "q3", "q4", "q5", "q6",
    "q7", "q8", "q9", "q10", "deja", "ante", "han", "estan", "pierde", "ha", "dia", "50", "2022",
    "desde", "despues", "ano", "dias", "otra", "luego", "km", "pedro", "2024", "2025", "2023", "hace",
    "donde", "otro", "daã", "iba", "les", "dan", "45", "tienen", "hacen", "juan"
]

vectorizer = CountVectorizer(max_features=5000, stop_words=spanish_stopwords)

In [ ]:
# Fill NaN values with an empty string
data["titulo"] = data["titulo"].fillna("").astype(str)
data["subtitular"] = data["subtitular"].fillna("").astype(str)

data["text"] = data["titulo"] + " " + data["subtitular"]

In [ ]:
# Download Spanish stopwords (only needed once)
#nltk.download("stopwords")
#spanish_stopwords = stopwords.words("spanish")

# Define range of parameters to test
n_components_range = [5, 10, 15, 20]  # Example LDA n_components values
num_classes_range = [5, 10, 15, 20]  # Example LCA num_classes values

# Prepare an empty list to store the latent class distributions
distributions = []

for n_components in n_components_range:
    for num_classes in num_classes_range:
        # Apply LDA with Spanish stop words
        vectorizer = CountVectorizer(max_features=5000, stop_words=spanish_stopwords)
        X = vectorizer.fit_transform(data["text"])

        lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
        X_topics = lda.fit_transform(X)
        data["topic"] = X_topics.argmax(axis=1)

        # Apply OneHotEncoder
        encoder = OneHotEncoder(sparse_output=False)
        X_lca = encoder.fit_transform(data[["topic"]])

        # Apply LCA (GaussianMixture)
        lca_model = GaussianMixture(n_components=num_classes, covariance_type="full", random_state=42)
        data["latent_class"] = lca_model.fit_predict(X_lca)

        # Calculate class distribution
        class_distribution = data["latent_class"].value_counts().values
        distributions.append([n_components, num_classes, class_distribution])

# Convert results into a DataFrame for visualization
distributions_df = pd.DataFrame(distributions, columns=["n_components", "num_classes", "class_distribution"])

# Make sure all classes are represented in each distribution
distributions_df["class_count"] = distributions_df["class_distribution"].apply(lambda x: np.sum(x))


In [ ]:
# Define a broader range of random states (e.g., 20 values between 0 and 3000)
random_state_range = np.linspace(0, 3000, dtype=int)

# Convert class distributions into DataFrame
plot_data = []

for entry in distributions:
    random_state, num_classes, class_distribution = entry  # Ensure distributions include random_state
    for class_idx, count in enumerate(class_distribution):
        plot_data.append([random_state, num_classes, class_idx, count])

# Create DataFrame
plot_df = pd.DataFrame(plot_data, columns=["random_state", "num_classes", "class_idx", "count"])

# Plot distributions for each num_classes set
plt.figure(figsize=(36, 8))
sns.set_style("whitegrid")

for idx, num_classes in enumerate(num_classes_range):
    plt.subplot(1, len(num_classes_range), idx + 1)
    subset = plot_df[plot_df["num_classes"] == num_classes]
    sns.lineplot(
        x="random_state", y="count", hue="class_idx", data=subset, marker="o", palette="tab10"
    )
    plt.title(f"Num Classes: {num_classes}")
    plt.xlabel("Random State")
    plt.ylabel("Class Count")
    plt.legend(title="Latent Class", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

plt.show()

In [ ]:
bics = []
aics = []
class_range = range(2, 30)  # Try between 2 and 10 classes

for n in class_range:
    gmm = GaussianMixture(n_components=19, covariance_type="tied", random_state=50)
    gmm.fit(X_lca)
    bics.append(gmm.bic(X_lca))
    aics.append(gmm.aic(X_lca))

bic_scores = class_range[np.argmin(bics)]
print(f"Optimal number of classes based on BIC: {bic_scores}")


In [ ]:
# Standardize data
scaler = StandardScaler()
X_lca_scaled = scaler.fit_transform(X_lca)

# Define range of latent classes to test
class_range = range(2, 30)  # Try between 2 and 30 classes

# Store results
aic_scores = []
entropy_scores = []

def entropy(probs):
    return -np.sum(probs * np.log(probs + 1e-10), axis=1).mean()

for n in class_range:
    lca_model = GaussianMixture(n_components=n, covariance_type="spherical", init_params="kmeans", random_state=25)
    lca_model.fit(X_lca_scaled)

    # Compute AIC
    aic_scores.append(lca_model.aic(X_lca_scaled))

    # Compute Entropy
    class_probs = lca_model.predict_proba(X_lca_scaled)
    entropy_scores.append(entropy(class_probs))

# Plot AIC and Entropy
fig, ax1 = plt.subplots()

ax1.set_xlabel("Number of Classes")
ax1.set_ylabel("AIC", color="tab:blue")
ax1.plot(class_range, aic_scores, marker="o", color="tab:blue", label="AIC")
ax1.tick_params(axis="y", labelcolor="tab:blue")

ax2 = ax1.twinx()
ax2.set_ylabel("Entropy", color="tab:red")
ax2.plot(class_range, entropy_scores, marker="s", color="tab:red", label="Entropy")
ax2.tick_params(axis="y", labelcolor="tab:red")

fig.tight_layout()
plt.title("AIC vs Entropy for Different Class Counts")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_lca_scaled = scaler.fit_transform(X_lca)

# Define number of latent classes
num_classes = 12

# Fit LCA model using Gaussian Mixture Model (GMM)
lca_model = GaussianMixture(n_components=num_classes, covariance_type="spherical", init_params="kmeans", random_state=25)
data["latent_class"] = lca_model.fit_predict(X_lca)

# Check class distribution
print(data["latent_class"].value_counts())

In [ ]:
def entropy(probs):
    return -np.sum(probs * np.log(probs + 1e-10), axis=1).mean()

class_probs = lca_model.predict_proba(X_lca)
print("LCA Model Entropy:", entropy(class_probs))

# Get AIC value
aic_value = lca_model.aic(X_lca_scaled)
print(f"AIC: {aic_value}")

In [ ]:
sns.countplot(x=data["latent_class"])
plt.title("Distribution of Latent Classes")
plt.show()

In [ ]:
print(data.groupby("latent_class")["topic"].value_counts())

In [ ]:
# Function to extract top words per topic without duplication
def get_top_words_unique(lda_model, feature_names, num_words=50):
    topic_keywords = {}
    used_keywords = set()  # Keep track of already used keywords
    for topic_idx, topic in enumerate(lda_model.components_):
        top_keywords = []
        for i in topic.argsort()[-num_words:]:
            keyword = feature_names[i]
            if keyword not in used_keywords:
                top_keywords.append(keyword)
                used_keywords.add(keyword)
        
        # Ensure we have enough unique keywords, if not, add more from the remaining options
        if len(top_keywords) < num_words:
            for i in topic.argsort()[:-num_words-1:-1]:
                keyword = feature_names[i]
                if keyword not in used_keywords:
                    top_keywords.append(keyword)
                    used_keywords.add(keyword)
        
        topic_keywords[topic_idx] = top_keywords
    return topic_keywords

# Get words for each topic ensuring no duplication
topic_keywords = get_top_words_unique(lda, feature_names)

# Display top words per topic
for topic, words in topic_keywords.items():
    print(f"Topic {topic}: {', '.join(words)}")



In [ ]:
probs = lca_model.predict_proba(X_lca_scaled)
print(probs.min(), probs.max())

# Esta es la conexion del modelo

In [ ]:
new_title = "colisionan y vuelcan"
new_subtitle = "familia se dirigia a vender al mercado"

# Merge text
new_text = new_title + " " + new_subtitle

# Transform into vector
X_new = vectorizer.transform([new_text])
X_new_topic = lda.transform(X_new).argmax(axis=1)  # Get topic

# Convert topic into categorical format
X_new_lca = encoder.transform([[X_new_topic[0]]])

# Predict latent class
predicted_class = lca_model.predict(X_new_lca)
print("Predicted Latent Class:", predicted_class[0])

# FINAL

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Define category-related keywords with weights
category_keywords = {
    "Religión y Tradición": ["tradicional", "cortejo", "nazareno", "mayor", "comienzan", "independencia", "imagen", "acompanan", "conmemoran", "patrona", "feligreses", "cuaresma", "recorre", "concepcion", "catolicos", "historico", "recorrido", "domingo", "mercado", "cientos", "estudiantes", "procesion", "antigua", "festejos", "participan", "preparan", "devotos", "fiestas", "actividades", "visitantes", "virgen", "celebran"],
    "Criminalidad y Violencia": ["cerca", "asesinato", "nina", "mecanico", "esperaron", "orfandad", "hermosa", "problema", "sicarios", "nino", "desconocidos", "atacan", "bala", "eeuu", "asesinan", "atacado", "balazos", "menor"],
    "Accidentes Viales": ["dirigia", "embestido", "salio", "familiares", "milagro", "hija", "murio", "frenos", "fallecio", "salvan", "hijos", "salen", "barranco", "carretera", "morir", "quedo", "amigos", "auto", "madre", "frente", "hondonada", "arrollado", "iban", "carro", "impacta", "choca", "metros", "cae"],
    "Proyectos de Infraestructura": ["santiago", "nuevo", "mareros", "fuerzas", "seguridad", "adultos", "herido", "mexico", "centro", "sale", "muerto", "hospital", "grave", "bus", "fiesta", "resultan", "victima", "regresaba", "heridos", "asalto", "pnc", "vehiculo", "sobrevive", "herida", "ser", "fuego", "accidente", "disparan", "mueren", "muere", "tragedia", "familia", "victimas", "armado", "ataque"],
    "Demandas Sociales": ["quedar", "sur", "ayuda", "casco", "sigue", "casi", "tierra", "construir", "drenaje", "pavimentacion", "zonas", "cierre", "via", "urbano", "podria", "reparacion", "invierno", "viven", "transporte", "colapsar", "comuna", "tramo", "proyecto", "reparan"],
    "Riesgos y Emergencias": ["afectadas", "provisional", "alternas", "perros", "afecta", "habilitan", "ocasiona", "vivienda", "punto", "complica", "vias", "villa", "ciudad", "hacia", "socavamiento", "carril", "barrio", "acceso", "derrumbe", "construccion", "dano", "desnivel", "hundimiento", "colapso", "kilometro"],
    "Inseguridad y Conflictos": ["carreteras", "camino", "fin", "siguen", "temen", "recibir", "guatemala", "falta", "pueden", "esta", "cosechas", "autoridades", "primera", "chimaltenango", "debido", "lago", "muro", "dejan", "coatepeque", "riesgo", "aldeas", "partido", "pierden", "derrumbes", "agricultores", "incendio", "semana", "paso", "afectan", "locales", "transportistas", "perdidas", "oficial", "nacional", "municipal", "millones", "evitar", "vehicular", "conductores", "peten", "ruta", "piden"],
    "Seguridad y Bienestar": ["viviendas", "lluvias", "costo", "guatemaltecos", "abandono", "rafael", "dios", "comerciantes", "seran", "comunidades", "vuelven", "jesus", "sufren", "todo", "salud", "casos", "fe", "vehiculos", "municipio", "reciben", "esperan", "centros", "menos", "personas", "ninos", "vendedores", "fieles", "devocion", "obra", "municipios", "horas", "llegar", "patrono", "agua", "antonio", "bloqueos", "trabajos", "flores", "familias", "miguel", "cada", "francisco", "afectados", "pobladores", "jose", "aldea", "vecinos", "marcos", "sacatepequez", "san"],
    "Desastres Naturales": ["tendran", "cayo", "final", "hallado", "pueblo", "capital", "descenso", "nivel", "arriesgan", "quebrada", "rural", "vez", "habitantes", "lluvia", "tratamiento", "mundo", "precio", "area", "afluente", "desborde", "julia", "planta", "residentes", "caudal", "crecida", "rios", "motagua"],
    "Accidentes y Tragedias": ["izabal", "puerto", "buscan", "tenia", "vecino", "agricultor", "despiden", "julio", "tormenta", "hectareas", "registran", "norte", "sido", "sepultan", "supuestos", "prevenir", "dejado", "mama", "ocurrido", "80", "joven", "pais", "forestales", "barrios", "departamento", "policia", "inundaciones", "asesinado", "incendios", "dengue"],
    "Crimenes": ["lleva", "turistas", "servicio", "caminan", "clandestinos", "cementerio", "christi", "temporada", "desfile", "honor", "patronal", "comunidad", "ana", "lugar", "tiene", "material", "pasado", "limpian", "tradicion", "mal", "calles", "estado", "basureros", "llevan", "contaminacion", "atencion", "exigen", "lodo", "negro", "toneladas", "cristo", "desechos", "feria", "basura"],
    "Educación y Salud Comunitaria": ["drenajes", "educativos", "solo", "labores", "instalaciones", "agresion", "sexual", "atienden", "docentes", "ninas", "colapsa", "negras", "aguas", "techo", "colision", "ciclo", "escolar", "pacientes", "aulas", "maestros", "padres", "alumnos", "clases", "escuela"],
}

# Convert category keywords into a feature vector
def get_weighted_features(text):
    words = text.lower().split()
    category_scores = {category: 0 for category in category_keywords}
    
    for category, keywords in category_keywords.items():
        for word in words:
            if word in keywords:
                category_scores[category] += 1  # Assign a weight based on word presence
    
    return np.array(list(category_scores.values()))

# Example usage
new_title = ""
new_subtitle = "reparacion negro pacientes aldea puente paso arrollado hombre muere hospital"
new_text = new_title + " " + new_subtitle
features = get_weighted_features(new_text)
print("Feature Vector:", features)

In [ ]:
# Function to Categorize New Texts

def categorize_new_texts(new_data, vectorizer, lda_model, lca_model, encoder):
    """
    Categorizes new newspaper texts based on trained LDA and LCA models.
    
    Parameters:
    new_data (pd.DataFrame): A DataFrame with "titulo" and "subtitular" columns.
    vectorizer (CountVectorizer): Trained CountVectorizer.
    lda_model (LatentDirichletAllocation): Trained LDA model.
    lca_model (GaussianMixture): Trained LCA model.
    encoder (OneHotEncoder): Trained encoder for LCA.

    Returns:
    pd.DataFrame: The new data with assigned topic and latent class.
    """
    # Ensure text columns are strings and concatenate
    new_data["titulo"] = new_data["titulo"].fillna("").astype(str)
    new_data["subtitular"] = new_data["subtitular"].fillna("").astype(str)
    new_data["text"] = new_data["titulo"] + " " + new_data["subtitular"]

    # Transform text using the existing vectorizer
    X_new = vectorizer.transform(new_data["text"])
    
    # Predict topics using LDA
    new_data["topic"] = lda_model.transform(X_new).argmax(axis=1)

    # Encode topic for LCA
    X_new_encoded = encoder.transform(new_data[["topic"]])
    
    # Predict latent classes using LCA
    new_data["latent_class"] = lca_model.predict(X_new_encoded)
    
    return new_data

# Load new data
new_texts = pd.read_csv("categories.csv")  # Path to your CSV file

# Categorize new texts
categorized_texts = categorize_new_texts(new_texts, vectorizer, lda, lca_model, encoder)

# Print results
print(categorized_texts[["titulo", "subtitular", "categoria", "topic", "latent_class"]].head())


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the CSV
df = pd.read_csv('categories.csv')

# View the first few rows of the dataset
df.head()

# Load BETO tokenizer and model
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', num_labels=len(df['categoria'].unique()))

# Preprocess the title and subtitle into the format the model expects
def preprocess_text(row):
    # Ensure both title and subtitle are strings (handle NaN)
    title = str(row['titulo']) if pd.notnull(row['titulo']) else ''
    subtitle = str(row['subtitular']) if pd.notnull(row['subtitular']) else ''
    
    # Concatenate title and subtitle
    text = title + " " + subtitle
    return tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Apply preprocessing to your data
df['inputs'] = df.apply(preprocess_text, axis=1)

def classify_newspaper(text):
    # Preprocess the input text (new title and subtitle)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Get the predicted category
    predicted_class = torch.argmax(logits, dim=1).item()
    return df['categoria'].unique()[predicted_class]

# Test the classification with a new title and subtitle
titulo = "Carro choca en media via"
subtitulo = "15 personas mueren en golpe de autobus luego de pasar por la carretera las fuentes"
new_text = titulo + " " + subtitulo
print(classify_newspaper(new_text))

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Inseguridad y Conflictos
